In [ ]:
from carbonplan_buffer_analysis.prefect.tasks import ravg

import geopandas
import fsspec

import json

import rioxarray

import matplotlib.pyplot as plt

import pandas as pd

from carbonplan_styles.mpl.colormaps import fire_light
from carbonplan_styles.colors import light

# indirectly from matplotlib
from matplotlib.cm import get_cmap

fire_cmap = get_cmap("fire_light")

plt.rc("font", **{"size": 15})

In [ ]:
import numpy as np
import matplotlib
from matplotlib.colors import BoundaryNorm

import matplotlib

In [ ]:
da = ravg.load_ravg.run("bootleg")
subset = ravg.get_ravg_subset.run(da, "ACR273")
acres_burned = ravg.get_ravg_counts.run(subset)

In [ ]:
rpj = da.rio.reproject("epsg:4326")

In [ ]:
with fsspec.open(
    "https://storage.googleapis.com/carbonplan-forest-offsets/carb-geometries/raw/ACR273.json"
) as f:
    d = json.load(f)

shp = geopandas.GeoDataFrame.from_features(d)

shp = shp.set_crs("epsg:4326")

shp.geometry = shp.simplify(0.0001)

In [ ]:
tp_da = rpj.where((rpj > 0) & (rpj < 8))

In [ ]:
norm = matplotlib.colors.Normalize(vmin=0, vmax=tp_da.max())


shifted_cmap = matplotlib.colors.ListedColormap(
    [fire_cmap(norm(x)) for x in np.linspace(1, 7, 7)]
)

In [ ]:
hist_data = pd.Series(acres_burned)


tp_hist = (hist_data / 1_000).sort_index()

tp_hist.index = tp_hist.index.astype(int)

In [ ]:
f, ax = plt.subplots(
    1, 2, gridspec_kw={"width_ratios": [3, 1]}, figsize=(7, 3.5)
)
orig_aspect = ax[0].get_aspect()

# plt.subplot2grid(shape=(3,3))
tp_da.plot(add_colorbar=False, ax=ax[0], cmap=shifted_cmap, rasterized=True)

shp.plot(ax=ax[0], color=light["secondary"], alpha=0.85, rasterized=True)
ax[0].set_aspect(orig_aspect)
ax[0].set_title("")
# ax[0].spines['top'].set_visible(False)
# ax[0].spines['right'].set_visible(False)

# ax[0].xaxis.set_ticks_position('bottom')
# plt.axis('off')
for _, spine in ax[0].spines.items():
    spine.set_visible(False)
# ax[0].axis('off')

ax[0].tick_params(bottom=True, top=True, left=True, right=True, direction="out")
ax[0].tick_params("both", length=7, width=1, which="major")

ax[0].set_xticks([-121.6, -121.1, -120.6])
ax[0].set_xlim(-121.6 - 0.05, -120.6 + 0.05)
ax[0].set_xticklabels([])

ax[0].annotate(
    text=f"{ax[0].get_xticks()[0]}°",
    c=light["secondary"],
    fontsize=12,
    xy=(0.07, 0.99),
    xycoords="axes fraction",
)
ax[0].annotate(
    text=f"{ax[0].get_xticks()[1]}°",
    c=light["secondary"],
    fontsize=12,
    xy=(0.52, 0.99),
    xycoords="axes fraction",
)
ax[0].annotate(
    text=f"{ax[0].get_xticks()[2]}°",
    c=light["secondary"],
    fontsize=12,
    xy=(0.975, 0.99),
    xycoords="axes fraction",
)


ax[0].set_yticks([42.4, 42.7, 43])
ax[0].set_ylim(42.4 - 0.05, 43 + 0.05)
ax[0].set_yticklabels([])

ax[0].annotate(
    text=f"{ax[0].get_yticks()[2]}°",
    c=light["secondary"],
    fontsize=12,
    xy=(-0.025, 0.85),
    xycoords="axes fraction",
)
ax[0].annotate(
    text=f"{ax[0].get_yticks()[1]}°",
    c=light["secondary"],
    fontsize=12,
    xy=(-0.025, 0.425),
    xycoords="axes fraction",
)
# ax[0].annotate(text=f'{ax[0].get_yticks()[0]}°', c=light['secondary'], fontsize=12,
#               xy=(-0.025,0.05), xycoords='axes fraction')

ax[0].set_ylabel(None)
ax[0].set_xlabel(None)


tp_hist.plot.bar(ax=ax[1], color=shifted_cmap.colors, grid=False)
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=0)

ax[1].spines["top"].set_visible(False)
ax[1].spines["right"].set_visible(False)
ax[1].set_ylabel("Project Area\n(thousand acres)")
ax[1].set_xlabel("Mortality Class", labelpad=10)
ax[1].set_yticks([0, 25, 50])
ax[1].set_ylim(0, 50)


plt.subplots_adjust(wspace=0.65)
plt.savefig("../img/bootleg.pdf", bbox_inches="tight", dpi=300)

In [ ]:
f, ax = plt.subplots(
    1, 2, gridspec_kw={"width_ratios": [3, 1]}, figsize=(7, 3.5)
)
orig_aspect = ax[0].get_aspect()

# plt.subplot2grid(shape=(3,3))
tp_da.plot(add_colorbar=False, ax=ax[0], cmap=shifted_cmap, rasterized=True)

shp.plot(ax=ax[0], color=light["secondary"], alpha=0.85, rasterized=True)
ax[0].set_aspect(orig_aspect)
ax[0].set_title("")
# ax[0].spines['top'].set_visible(False)
# ax[0].spines['right'].set_visible(False)

# ax[0].xaxis.set_ticks_position('bottom')
# plt.axis('off')
for _, spine in ax[0].spines.items():
    spine.set_visible(False)
# ax[0].axis('off')
# ax[0].grid()

# ax[0].xaxis.set_ticks_position("none")
# ax[0].yaxis.set_ticks_position("none")

ax[0].tick_params(bottom=True, top=True, left=True, right=True, direction="out")
ax[0].tick_params("both", length=7, width=1, which="major")

xmin = -122.3
ax[0].set_xticks([xmin, -121.95, -121.6])
ax[0].set_xlim([xmin - 0.05, -121.6 + 0.05])
ax[0].set_xticklabels([])

ax[0].annotate(
    text=f"{ax[0].get_xticks()[0]}°",
    c=light["secondary"],
    fontsize=12,
    xy=(0.09, 0.99),
    xycoords="axes fraction",
)
ax[0].annotate(
    text=f"{ax[0].get_xticks()[1]}°",
    c=light["secondary"],
    fontsize=12,
    xy=(0.52, 0.99),
    xycoords="axes fraction",
)
ax[0].annotate(
    text=f"{ax[0].get_xticks()[2]}°",
    c=light["secondary"],
    fontsize=12,
    xy=(0.96, 0.99),
    xycoords="axes fraction",
)

ax[0].set_yticks([44.6, 44.75, 44.9])
ax[0].set_ylim([44.6 - 0.05, 44.9 + 0.05])
ax[0].set_yticklabels([])

ax[0].annotate(
    text=f"{ax[0].get_yticks()[2]}°",
    c=light["secondary"],
    fontsize=12,
    xy=(-0.025, 0.8),
    xycoords="axes fraction",
)
ax[0].annotate(
    text=f"{ax[0].get_yticks()[1]}°",
    c=light["secondary"],
    fontsize=12,
    xy=(-0.025, 0.425),
    xycoords="axes fraction",
)
ax[0].annotate(
    text=f"{ax[0].get_yticks()[0]}°",
    c=light["secondary"],
    fontsize=12,
    xy=(-0.025, 0.05),
    xycoords="axes fraction",
)

# ax[0].spines['bottom'].set_position(('outward', 10))
# ax[0].spines['left'].set_position(('outward', 10))
# ax[0].set_yticks([42, 42.5, 43])
# ax[0].set_ylim(42,43)
ax[0].set_ylabel(None)
ax[0].set_xlabel(None)

# ax[0].set_xticks([-121.5, -121, -120.5])
# ax[0].set_xlim(-121.5, -120.5)


# cb.ax.tick_params(size=0)
tp_hist.plot.bar(ax=ax[1], color=shifted_cmap.colors, grid=False)
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=0)

ax[1].spines["top"].set_visible(False)
ax[1].spines["right"].set_visible(False)
ax[1].set_ylabel("Project Area\n(thousand acres)")
ax[1].set_xlabel("Mortality Class", labelpad=10)
# ax[1].set_yticks([0,25,50])
ax[1].set_ylim(0, 8)


plt.subplots_adjust(wspace=0.65)
plt.savefig("../img/lionshead.pdf", bbox_inches="tight", dpi=300)